## WorldView Stereopair Processing: SpaceNet Atlanta Example

This notebook demonstrates a stereo processing workflow for WorldView satellite imagery using the NASA Ames Stereo Pipeline (ASP).

The example uses publicly available WorldView-2 data from the [SpaceNet Atlanta dataset](https://spacenet.ai/off-nadir-building-detection/), which provides multi-view imagery at various off-nadir angles over Atlanta, Georgia.

---

### Processing Overview

This notebook covers a basic stereo workflow:

1. **Data Retrieval** - Download WorldView imagery from AWS S3
2. **CCD Artifact Correction** - Apply `wv_correct` to fix sensor artifacts
3. **Reference DEM Preparation** - Obtain and prepare Copernicus 30m DEM
4. **Define Crop Windows** - Select a region of interest for processing
5. **Bundle Adjustment** - Camera model refinement
6. **Mapprojection** - Project cropped images onto reference DEM
7. **Stereo Processing** - Generate DEM from stereopair
8. **Visualization with `asp_plot`** - Analyze results and compare with ICESat-2

**Note:** The full SpaceNet images are very large (~35,000 x 55,000 pixels each). To make processing manageable, we crop to a smaller region of interest during mapprojection.

For advanced workflows including jitter correction, see [ASP Documentation Section 16.39.9](https://stereopipeline.readthedocs.io/en/latest/tools/jitter_solve.html#example-2-worldview-3-digitalglobe-images-on-earth).

---

## 1. Data Retrieval

### SpaceNet Atlanta Dataset

The [SpaceNet Off-Nadir Building Detection Challenge](https://spacenet.ai/off-nadir-building-detection/) provides WorldView-2 imagery collected over Atlanta, Georgia at multiple viewing angles. This dataset is ideal for demonstrating stereo processing because it includes imagery from various off-nadir angles that can form stereopairs.

The data is hosted on AWS S3 and can be accessed directly via HTTPS without authentication. Available scenes are organized by nadir angle and catalog ID (CATID):

```
nadir7_catid_1030010003D22F00/
nadir8_catid_10300100023BC100/
nadir10_catid_1030010003993E00/
...
nadir52_catid_1030010003BDDC00/
nadir53_catid_1030010003193D00/
```

### Selecting a Stereopair

For stereo processing, we want two images with:
- Sufficient baseline (convergence angle ~15-45 degrees)
- Good overlap
- Similar illumination conditions

For this example, we select:
- **Image 1**: `nadir7_catid_1030010003D22F00` (near-nadir, ~7 degrees off-nadir)
- **Image 2**: `nadir30_catid_10300100036D5200` (off-nadir, ~30 degrees off-nadir)

This pairing provides a convergence angle of approximately 23 degrees, suitable for stereo reconstruction.

### Download Commands

Create a working directory and download only the required files (panchromatic imagery and XML camera models) using `wget`:

```bash
# Create working directory structure
mkdir -p atlanta_stereo
cd atlanta_stereo

# Base URL for SpaceNet Atlanta data
base_url="https://spacenet-dataset.s3.amazonaws.com/AOIs/AOI_6_Atlanta"

# Download nadir7 scene (Image 1) - PAN image and XML only
wget -O img1.tif "${base_url}/nadir7_catid_1030010003D22F00/PAN/AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.tif"
wget -O img1.xml "${base_url}/nadir7_catid_1030010003D22F00/PAN/AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.XML"

# Download nadir30 scene (Image 2) - PAN image and XML only
wget -O img2.tif "${base_url}/nadir30_catid_10300100036D5200/PAN/AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.tif"
wget -O img2.xml "${base_url}/nadir30_catid_10300100036D5200/PAN/AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.XML"



# Base URL for SpaceNet Atlanta data
base_url="https://spacenet-dataset.s3.amazonaws.com/AOIs/AOI_6_Atlanta"

# Download nadir7 scene (Image 1) - PAN image and XML only
wget "${base_url}/nadir7_catid_1030010003D22F00/PAN/AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.tif"
wget "${base_url}/nadir7_catid_1030010003D22F00/PAN/AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.XML"

# Download nadir30 scene (Image 2) - PAN image and XML only
wget "${base_url}/nadir30_catid_10300100036D5200/PAN/AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.tif"
wget "${base_url}/nadir30_catid_10300100036D5200/PAN/AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.XML"


stereo_gui --threads 8 AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.tif AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.tif AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.XML AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.XML
```




**Note:** The downloads are ~4 GB each for the panchromatic images. We rename the files to `img1.*` and `img2.*` for convenience in subsequent commands.

Each scene contains:
- `*_PAN.tif` - Panchromatic imagery (~0.5m GSD)
- `*_PAN.XML` - Camera model metadata (RPC coefficients)

For stereo processing, we use the panchromatic imagery and corresponding XML files.

---

## 2. CCD Artifact Correction

WorldView sensors have subpixel misalignments between CCD arrays that can cause discontinuities in DEMs. The [`wv_correct`](https://stereopipeline.readthedocs.io/en/latest/tools/wv_correct.html) tool corrects these artifacts.

**Note:** WorldView-2 imagery processed after May 26, 2022 has reduced CCD artifacts, and applying `wv_correct` may make results worse. ASP 3.3.0+ automatically detects this and skips correction. The SpaceNet Atlanta data is from 2009, so correction is recommended.

```bash
# Correct CCD artifacts in image 1
wv_correct img1.tif img1.xml img1_corr.tif

# Correct CCD artifacts in image 2
wv_correct img2.tif img2.xml img2_corr.tif

# Assign the cameras
cp img1.xml img1_corr.xml
cp img2.xml img2_corr.xml
```

The corrected images will be used for all subsequent processing steps.

---

## 3. Reference DEM Preparation

A reference DEM is required for:
- Mapprojecting the input images
- Validating the output DEM

We use the Copernicus 30m GLO-30 DEM, which provides global coverage with good accuracy.

### Download Copernicus DEM

Using [fetch_dem](https://github.com/uw-cryo/fetch_dem) (requires an [OpenTopography API key](https://portal.opentopography.org/requestService?service=api)):

```bash
# Get DEM for the full image extent
python download_global_DEM.py \
  -demtype COP30 \
  -raster_fn img1_corr.tif \
  -out_fn cop30_atlanta.tif \
  -apikey d34347b294d0570fc27f959e452bfd47
```

### Adjust Vertical Reference

The Copernicus DEM uses EGM2008 geoid heights. ASP requires heights relative to the WGS84 ellipsoid:

```bash
# Convert from EGM2008 geoid to WGS84 ellipsoid heights
dem_geoid --geoid egm2008 --reverse-adjustment cop30_atlanta.tif

# Rename for clarity
mv cop30_atlanta-adj.tif ref.tif
```

### Reproject

Reproject the DEM to UTM, as all subsequent processing will be in UTM

```bash
proj="+proj=utm +zone=16 +datum=WGS84 +units=m +no_defs"

gdalwarp -r cubic -t_srs "$proj" ref.tif ref-utm.tif
```

---

## 4. Define Region of Interest

The full SpaceNet images are very large. To make processing tractable, we define a region of interest (ROI) to process.

You can determine the overlapping region interactively using `stereo_gui`:

```bash
stereo_gui img1_corr.tif img2_corr.tif img1_corr.xml img2_corr.xml
```

Draw a bounding box on the overlap area and note the coordinates. The `mapproject` tool accepts geographic coordinates with `--t_projwin` (format: `xmin ymin xmax ymax` in the target projection).

For this example, we define a common ROI in UTM coordinates that covers the overlapping area (~4 km x 4.5 km):

```bash
# Define region of interest in UTM Zone 16N coordinates (xmin ymin xmax ymax)
# This covers the overlap between both images
projwin="734233 3725600 738135 3729655"

# projwin="736375 3728430 738555 3726250"

# projwin="736375 3726250 738555 3728430"
```

Using a single geographic/projected bounding box is simpler than pixel coordinates because:
- The same ROI works for both images
- `mapproject` automatically determines which source pixels to read
- Easier to define regions based on real-world locations

**Alternative:** You can also use longitude/latitude with appropriate projection handling, or define ROIs by center point and dimensions.

---

## 5. Bundle Adjustment

Bundle adjustment refines camera models by minimizing reprojection errors of matched feature points. This step improves stereo correlation quality.

Bundle adjustment must be run on the full images, un-cropped:

```bash
bundle_adjust \
  --threads 24 \
  --ip-detect-method 1 \
  --ip-per-image 10000 \
  --tri-weight 0.1 \
  --tri-robust-threshold 0.1 \
  --camera-weight 0 \
  --remove-outliers-params '75.0 3.0 20 20' \
  img1_corr.tif img2_corr.tif \
  img1_corr.xml img2_corr.xml \
  -o ba/run

# bundle_adjust \
#   --threads 24 \
#   --ip-detect-method 1 \
#   --ip-per-image 10000 \
#   --tri-weight 0.1 \
#   --tri-robust-threshold 0.1 \
#   --camera-weight 0 \
#   --remove-outliers-params '75.0 3.0 20 20' \
#   AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.tif AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.tif \
#   AOI_6_Atlanta_Atlanta_nadir7_catid_1030010003D22F00_058222361010_01_assembly_PAN.XML AOI_6_Atlanta_Atlanta_nadir30_catid_10300100036D5200_058332886010_01_assembly_PAN.XML \
#   -o ba_original_scenes/run
```

**Key parameters:**
- `--ip-detect-method 1`: Use SIFT-like interest point detector
- `--ip-per-image 10000`: Detect many interest points for robust matching
- `--tri-weight 0.1`: Weight for triangulation consistency constraint
- `--camera-weight 0`: Allow cameras to move freely (no prior constraint)
- `--remove-outliers-params '75.0 3.0 20 20'`: Generous outlier filtering for urban scenes with imperfect geometry

---

## 6. Mapprojection

Mapprojection orthorectifies the images onto the reference DEM surface. This simplifies stereo correlation by removing most geometric distortions, leaving only parallax due to elevation differences.

We mapproject only the ROI using the bundle-adjusted cameras:

```bash
proj="+proj=utm +zone=16 +datum=WGS84 +units=m +no_defs"

# Mapproject image 1 to the ROI
mapproject \
  -t rpc \
  --processes 48 \
  --threads 24 \
  --tr 0.5 \
  --t_srs "$proj" \
  --bundle-adjust-prefix ba/run \
  ref-utm.tif \
  img1_corr.tif img1_corr.xml \
  img1_corr.map.tif

# Mapproject image 2 to the same ROI
mapproject \
  -t rpc \
  --processes 48 \
  --threads 24 \
  --tr 0.5 \
  --t_srs "$proj" \
  --bundle-adjust-prefix ba/run \
  ref-utm.tif \
  img2_corr.tif img2_corr.xml \
  img2_corr.map.tif





proj="+proj=utm +zone=16 +datum=WGS84 +units=m +no_defs"

# Mapproject image 1 to the ROI
mapproject \
  -t rpc \
  --processes 48 \
  --threads 24 \
  --tr 10 \
  --t_srs "$proj" \
  --bundle-adjust-prefix ba_original_scenes/run \
  ref.tif \
  img1.tif img1.xml \
  img1.map.tif

# Mapproject image 2 to the same ROI
mapproject \
  -t rpc \
  --processes 48 \
  --threads 24 \
  --tr 10 \
  --t_srs "$proj" \
  --bundle-adjust-prefix ba_no_wv_correct/run \
  ref.tif \
  img2.tif img2.xml \
  img2.map.tif

```

**Key parameters:**
- `--tr 0.5`: Output resolution of 0.5 meters (WorldView-2 panchromatic GSD)
- `--t_srs`: Target spatial reference system (UTM Zone 16N for Atlanta)
- `--t_projwin`: Region of interest in projected coordinates (xmin ymin xmax ymax)
- `--bundle-adjust-prefix`: Use the refined camera models from bundle adjustment

---

## 7. Stereo Processing

Run stereo correlation on the mapprojected (and cropped) images to generate a point cloud and DEM:

```bash
parallel_stereo \
  --stereo-algorithm asp_mgm \
  --subpixel-mode 9 \
  --processes 6 \
  --alignment-method none \
  --bundle-adjust-prefix ba/run \
  img1.map.tif img2.map.tif \
  img1.xml img2.xml \
  stereo/run \
  ref.tif
```

**Key parameters:**
- `--stereo-algorithm asp_mgm`: Use ASP's implementation of Semi-Global Matching
- `--subpixel-mode 9`: High-quality subpixel refinement
- `--alignment-method none`: Images are already aligned via mapprojection
- `--bundle-adjust-prefix ba/run`: Use bundle-adjusted camera models

### Generate DEM

```bash
proj="+proj=utm +zone=16 +datum=WGS84 +units=m +no_defs"

point2dem \
  --tr 0.5 \
  --t_srs "$proj" \
  --errorimage \
  stereo/run-PC.tif
```

The `--errorimage` flag produces an intersection error map showing triangulation consistency.

### Compute Difference vs Reference

```bash
geodiff \
  stereo/run-DEM.tif \
  ref.tif \
  -o stereo/run_vs_ref
```

---

## 7. Visualization with `asp_plot`

The sections below demonstrate using `asp_plot` to visualize and validate the stereo processing results.

### Setup

In [ ]:
# Set the base directory for your processing
directory = "/path/to/atlanta_stereo/"
bundle_adjust_directory = "ba/"
stereo_directory = "stereo/"
reference_dem = f"{directory}ref.tif"

In [ ]:
%load_ext autoreload
%autoreload 2

import contextily as ctx

In [ ]:
# Atlanta is in UTM Zone 16N
map_crs = "EPSG:32616"

ctx_kwargs = {
    "crs": map_crs,
    "source": ctx.providers.Esri.WorldImagery,
    "attribution_size": 0,
    "alpha": 0.5,
}

### Full Report Generation

Generate a comprehensive PDF report with a single CLI command:

In [ ]:
!asp_plot \
  --directory $directory \
  --bundle_adjust_directory $bundle_adjust_directory \
  --stereo_directory $stereo_directory \
  --reference_dem $reference_dem \
  --map_crs EPSG:32616 \
  --dem_gsd 0.5 \
  --subset_km 1 \
  --add_basemap True \
  --plot_icesat True \
  --plot_geometry True

---

## Individual Plots

The sections below demonstrate modular usage of `asp_plot` for detailed analysis.

### Processing Parameters

Extract and display the command-line parameters used during processing:

In [ ]:
from asp_plot.processing_parameters import ProcessingParameters

In [ ]:
processing_parameters = ProcessingParameters(
    processing_directory=directory,
    bundle_adjust_directory=bundle_adjust_directory,
    stereo_directory=stereo_directory
)
processing_parameters_dict = processing_parameters.from_log_files()

print(f"Processed on: {processing_parameters_dict['processing_timestamp']}\n")

print(f"Reference DEM: {processing_parameters_dict['reference_dem']}\n")

print(f"Bundle adjustment ({processing_parameters_dict['bundle_adjust_run_time']}):\n")
print(processing_parameters_dict["bundle_adjust"])

print(f"\nStereo ({processing_parameters_dict['stereo_run_time']}):\n")
print(processing_parameters_dict["stereo"])

print(f"\nPoint2dem ({processing_parameters_dict['point2dem_run_time']}):\n")
print(processing_parameters_dict["point2dem"])

### Scene Plots

Visualize the input satellite imagery:

In [ ]:
from asp_plot.scenes import ScenePlotter

In [ ]:
plotter = ScenePlotter(
  directory,
  stereo_directory,
  title="Input Scenes"
)

plotter.plot_scenes()

### Stereo Geometry

Analyze stereo acquisition geometry from XML camera metadata:

In [ ]:
from asp_plot.stereo_geometry import StereoGeometryPlotter

In [ ]:
geometry_plotter = StereoGeometryPlotter(
  directory
)

geometry_plotter.dg_geom_plot()

### Bundle Adjustment Residuals

Visualize bundle adjustment optimization results:

In [ ]:
from asp_plot.bundle_adjust import ReadBundleAdjustFiles, PlotBundleAdjustFiles

In [ ]:
ba_files = ReadBundleAdjustFiles(directory, bundle_adjust_directory)
resid_initial_gdf, resid_final_gdf = ba_files.get_initial_final_residuals_gdfs(residuals_in_meters=True)

In [ ]:
plotter = PlotBundleAdjustFiles(
  [resid_initial_gdf, resid_final_gdf],
  lognorm=True,
  title="Bundle Adjust Initial and Final Residuals (Log Scale)"
)

plotter.plot_n_gdfs(
    column_name="mean_residual",
    cbar_label="Mean residual (px)",
    map_crs=map_crs,
    **ctx_kwargs
)

In [ ]:
plotter.lognorm = False
plotter.title = "Bundle Adjust Initial and Final Residuals (Linear Scale)"

plotter.plot_n_gdfs(
    column_name="mean_residual",
    cbar_label="Mean residual (px)",
    common_clim=False,
    map_crs=map_crs,
    **ctx_kwargs
)

### Stereo DEM Results

Examine the stereo processing results:

In [ ]:
from asp_plot.stereo import StereoPlotter

In [ ]:
plotter = StereoPlotter(
  directory, 
  stereo_directory,
  reference_dem=reference_dem,
  dem_gsd=0.5,
)

plotter.title = "Stereo DEM Results"

plotter.plot_dem_results()

In [ ]:
plotter.title = "Hillshade with Details"

plotter.plot_detailed_hillshade(
  subset_km=1,
)

In [ ]:
plotter.title = "Stereo Match Points"

plotter.plot_match_points()

In [ ]:
plotter.title = "Disparity (meters)"

plotter.plot_disparity(
  unit="meters",
  quiver=True,
)

### ICESat-2 Altimetry Validation

Compare the ASP DEM with ICESat-2 ATL06-SR altimetry data:

In [ ]:
from datetime import datetime
from asp_plot.altimetry import Altimetry

In [ ]:
icesat = Altimetry(
  directory=directory,
  dem_fn=plotter.dem_fn
)

In [ ]:
# Request ATL06-SR data for multiple processing levels
icesat.request_atl06sr_multi_processing(
    processing_levels=["all", "ground"],
    save_to_parquet=True,
)

In [ ]:
# Filter out water bodies
icesat.filter_esa_worldcover(filter_out="water")

In [ ]:
# Apply temporal filters around the acquisition date
# SpaceNet Atlanta imagery was collected on December 22, 2009
icesat_filter_date = "2009-12-22"
icesat_filter_date = datetime.strptime(icesat_filter_date, "%Y-%m-%d").date()

icesat.predefined_temporal_filter_atl06sr(date=icesat_filter_date)

In [ ]:
# Map view of ICESat-2 vs DEM differences
icesat.mapview_plot_atl06sr_to_dem(
    key="ground_seasonal",
    map_crs=map_crs,
    **ctx_kwargs,
)

In [ ]:
# Histogram of DEM vs ICESat-2 differences
icesat.histogram(
    key="ground_seasonal",
    plot_aligned=False,
)

In [ ]:
# Run alignment report to assess DEM quality
icesat.alignment_report(
    processing_level="ground",
    minimum_points=100,
    agreement_threshold=0.25,
)

In [ ]:
icesat.alignment_report_df

---

## Next Steps

This notebook demonstrated a basic stereo workflow with cropped images. For more advanced processing:

1. **Full-resolution processing**: Remove the crop windows to process the entire image (requires significant compute resources)

2. **Jitter correction**: For highest quality DEMs, apply jitter correction following [ASP Documentation Section 16.39.9](https://stereopipeline.readthedocs.io/en/latest/tools/jitter_solve.html#example-2-worldview-3-digitalglobe-images-on-earth)

3. **DEM alignment**: Use `pc_align` to align the stereo DEM to ICESat-2 or the reference DEM for improved absolute accuracy

4. **Multi-view stereo**: Process additional image pairs from the SpaceNet dataset and merge the resulting DEMs